In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

os.chdir("/Users/jamesl/Workspace/titanic")
sns.set()

import data

# Reading the data
The first step will be to read in the data from data/raw/train.csv into a pandas dataframe. 

In [2]:
train = pd.read_csv("data/raw/train.csv")

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Looking for Missing Data and Taking a First Look
Next I will look for any missing data that we may have. Since most of the cabin information is missing, I may want to consider dropping that column. However, the letter infront of the cabin number indicates what deck someone was on in the titanic. I belive there may be a strong correlation between deck and survival and fare. However, I will explore that later. 

In [4]:
data.find_missing_values(train)

{'PassengerId': 0,
 'Survived': 0,
 'Pclass': 0,
 'Name': 0,
 'Sex': 0,
 'Age': 177,
 'SibSp': 0,
 'Parch': 0,
 'Ticket': 0,
 'Fare': 0,
 'Cabin': 687,
 'Embarked': 2}

In [5]:
train.loc[:, "Ticket"].sample(n=40)

759               110152
65                  2661
803                 2625
730                24160
396               350407
507               111427
772          S.O./P.P. 3
291                11967
355               345783
530                26360
617            A/5. 3336
488           A.5. 18509
361        SC/PARIS 2167
62                 36973
301               367226
351               113510
777               364516
310                11767
473      SC/AH Basle 541
588                14973
294               349233
504               110152
618               230136
421           A/5. 13032
860               350026
662                 5727
58            C.A. 34651
639               376564
191                28424
875                 2667
690                17474
796                17465
429    SOTON/O.Q. 392078
314         F.C.C. 13529
705               250655
674               239856
341                19950
43         SC/Paris 2123
469                 2666
313               349243


In [6]:
train.loc[:, "Name"].sample(n=40)

313                               Hendekovic, Mr. Ignjac
458                                  Toomey, Miss. Ellen
597                                  Johnson, Mr. Alfred
81                           Sheerlinck, Mr. Jan Baptist
389                                Lehmann, Miss. Bertha
546                    Beane, Mrs. Edward (Ethel Clarke)
819                         Skoog, Master. Karl Thorsten
101                     Petroff, Mr. Pastcho ("Pentcho")
161    Watt, Mrs. James (Elizabeth "Bessie" Inglis Mi...
827                                Mallet, Master. Andre
435                            Carter, Miss. Lucile Polk
323    Caldwell, Mrs. Albert Francis (Sylvia Mae Harb...
31        Spencer, Mrs. William Augustus (Marie Eugenie)
860                              Hansen, Mr. Claus Peter
171                                 Rice, Master. Arthur
38                    Vander Planke, Miss. Augusta Maria
516                         Lemore, Mrs. (Amelia Milley)
240                            

Every passenger seems to have a title attached to them. Next I will make a separate column containing their titles.

In [7]:
train['Title'] = data.find_title(train)